In [44]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import requests
from pandasql import sqldf
import json

In [13]:
cf_submissions_api = requests.get("https://codeforces.com/api/contest.status?contestId=1659&from=1")

In [14]:
submissions = cf_submissions_api.json()
df = pd.json_normalize(submissions,['result'])

In [21]:
df

,id,contestId,creationTimeSeconds,relativeTimeSeconds,programmingLanguage,verdict,testset,passedTestCount,timeConsumedMillis,memoryConsumedBytes,...,problem.points,problem.tags,author.contestId,author.members,author.participantType,author.ghost,author.startTimeSeconds,author.teamId,author.teamName,author.room
0,154063085,1659,1650366524,2147483647,GNU C++20 (64),OK,TESTS,49,343,3174400,...,1500.0,"[binary search, brute force, dp, greedy, imple...",1659,[{'handle': 'THEGodzilla'}],PRACTICE,False,1650206100,NaN,NaN,NaN
1,154063066,1659,1650366507,927,GNU C++17 (64),WRONG_ANSWER,TESTS,0,0,0,...,500.0,"[constructive algorithms, greedy, implementati...",1659,[{'handle': 'edgelord0'}],VIRTUAL,False,1650365580,NaN,NaN,NaN
2,154063048,1659,1650366484,2147483647,GNU C++17 (64),OK,TESTS,49,46,2457600,...,2000.0,"[constructive algorithms, data structures, gre...",1659,[{'handle': 'Utkarsh_Agarwal852'}],PRACTICE,False,1650206100,NaN,NaN,NaN
3,154063047,1659,1650366483,2147483647,Python 3,WRONG_ANSWER,TESTS,1,15,0,...,500.0,"[constructive algorithms, greedy, implementati...",1659,[{'handle': 'Nipun_Jugran'}],PRACTICE,False,1650206100,NaN,NaN,NaN
4,154063030,1659,1650366468,2147483647,GNU C++17,WRONG_ANSWER,TESTS,0,0,0,...,750.0,"[bitmasks, constructive algorithms, greedy, st...",1659,[{'handle': 'Ivan_Frost'}],PRACTICE,False,1650206100,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59962,153890450,1659,1650206183,83,GNU C++20 (64),WRONG_ANSWER,PRETESTS,0,0,0,...,500.0,"[constructive algorithms, greedy, implementati...",1659,[{'handle': 'BOOBA'}],CONTESTANT,False,1650206100,NaN,NaN,294.0
59963,153890449,1659,1650206182,82,GNU C++17,WRONG_ANSWER,PRETESTS,0,0,16076800,...,500.0,"[constructive algorithms, greedy, implementati...",1659,[{'handle': '-bankai-'}],CONTESTANT,False,1650206100,NaN,NaN,76.0
59964,153890444,1659,1650206173,73,GNU C11,COMPILATION_ERROR,PRETESTS,0,0,0,...,500.0,"[constructive algorithms, greedy, implementati...",1659,[{'handle': '-bankai-'}],CONTESTANT,False,1650206100,NaN,NaN,76.0
59965,153890436,1659,1650206162,62,GNU C++20 (64),WRONG_ANSWER,PRETESTS,0,0,0,...,500.0,"[constructive algorithms, greedy, implementati...",1659,[{'handle': 'BOOBA'}],CONTESTANT,False,1650206100,NaN,NaN,294.0


In [27]:
byTimeMemory = df[["contestId","problem.index","programmingLanguage","verdict","timeConsumedMillis", "memoryConsumedBytes"]]

In [28]:
byTimeMemory = byTimeMemory.rename(columns = {"problem.index":"problem_index"})

In [17]:
result = sqldf("select contestId, problem_index, programmingLanguage, timeConsumedMillis, memoryConsumedBytes from byTimeMemory where verdict = 'OK' ")

In [18]:
result.head(10)

,contestId,problem_index,programmingLanguage,timeConsumedMillis,memoryConsumedBytes
0,1659,C,GNU C++20 (64),343,3174400
1,1659,D,GNU C++17 (64),46,2457600
2,1659,B,GNU C++14,62,1024000
3,1659,B,GNU C++14,78,1331200
4,1659,A,GNU C++17,15,0
5,1659,C,GNU C++20 (64),31,1638400
6,1659,B,GNU C++14,93,1843200
7,1659,B,Python 2,186,4403200
8,1659,E,GNU C++17,1154,121241600
9,1659,E,GNU C++20 (64),311,118374400


In [30]:
result["memoryKB"] = result["memoryConsumedBytes"] // 1024
result = result.drop(columns = ["memoryConsumedBytes"])
result 

,contestId,problem_index,programmingLanguage,timeConsumedMillis,memoryKB
0,1659,C,GNU C++20 (64),343,3100
1,1659,D,GNU C++17 (64),46,2400
2,1659,B,GNU C++14,62,1000
3,1659,B,GNU C++14,78,1300
4,1659,A,GNU C++17,15,0
...,...,...,...,...,...
22978,1659,A,GNU C++20 (64),15,0
22979,1659,A,GNU C++17,31,0
22980,1659,A,GNU C++20 (64),15,0
22981,1659,A,GNU C++17 (64),15,0


In [47]:
result_groupby = result.groupby(by = ["contestId", "problem_index", "programmingLanguage"]).mean().round(1)

In [48]:
result_groupby

timeConsumedMillis  memoryKB
contestId problem_index programmingLanguage                                
1659      A             .NET Core C#                         88.8     800.0
                        Clang++17 Diagnostics                50.0     500.0
                        Delphi                               31.0       0.0
                        FPC                                 139.8     400.0
                        GNU C++14                            20.8    1189.3
...                                                           ...       ...
          F             GNU C++14                           149.4    6940.0
                        GNU C++17                           322.2    8423.1
                        GNU C++17 (64)                      200.6   11780.0
                        GNU C++20 (64)                      258.9   15685.0
                        GNU C11                             124.0    1800.0

[107 rows x 2 columns]

In [ ]:
result_groupby.to_csv("G:/CF782-17Apr2022-submissions.csv")